In [2]:
import glob
import os
import json

from lxml import etree

## Fabritius

In [3]:
tree = etree.parse('../data/fabritius/xml/fr.xml')

For now, we are only interested in the `Iconography` field in each record:

In [4]:
terms_ = set()
for record in list(tree.findall('record')):
    iconography = record.find('Iconography')
    if iconography is not None:
        for terms in iconography:
            terms_.add(terms.tag)
print(terms_)

{'iconographicTerms', 'generalSubjectDescription', 'iconographicInterpretation', 'subjectTerms', 'specificSubjectIdentification', 'conceptualTerms', 'subjectInterpretiveHistory'}


## Subject Terms

Let's start with the field `subjectTerms`:

In [5]:
def parse_lil_field(field):
    main_term, sub_terms = "<UNK>", []
    if '(' in field:
        idx = field.find('(')
        main_term = field[:idx].strip()
        if not main_term:
            main_term  = '<UNK>'
        sub_terms = field[idx:].strip().replace('(', '').replace(')', '')
        sub_terms = [st.strip() for st in sub_terms.split(';')]
        sub_terms = [sorted(st.split(' : ')) for st in sub_terms]
    else:
        main_term = field.strip()
    return main_term, sub_terms

def parse_subjectTerms(field):
    field = ' '.join(field.strip().split())
    
    fields_, d = [], []
    
    if ') ; ' in field:
        fields_ = [f.strip() for f in field.split(') ;')]
    elif ' ; ' in field and ')' not in field:
        fields_ = [f.strip() for f in field.split(' ; ')]
    else:
        fields_ = [field]
        
    fields = []
    for field in fields_:
        if ' ; ' in field and '(' not in field:
            fields.extend(field.split(' ; '))
        else:
            fields.append(field)
    
    d = []
    for field in fields:      
        main_term, sub_terms = parse_lil_field(field.strip())
        d.append({main_term : sub_terms})
    
    return d

def parse_xml_file(path):
    meta = {}
    tree = etree.parse(path)
    
    for record in list(tree.findall('record')):
        record_id = record.find('RecordID').text
        iconography = record.find('Iconography')

        if iconography is None:
            continue

        subject = iconography.find('subjectTerms')
        if subject is not None:
            d = parse_subjectTerms(subject.text)
            meta[record_id] = d
    return meta

In [6]:
for path in sorted(glob.glob('../data/fabritius/xml/*.xml')):
    print(path)
    meta = parse_xml_file(path)
    bn = os.path.basename(path).replace('.xml', '.json')
    with open(f'../data/fabritius/json/{bn}', 'w', encoding='utf8') as json_file:
        json.dump(meta, json_file, ensure_ascii=False, indent=4)

../data/fabritius/xml/en.xml
../data/fabritius/xml/fr.xml
../data/fabritius/xml/nl.xml


As such, the structured of a parsed record will be:
   - a ordered list of one-key dictionaries,
   - that have a "main descriptor" as single entry (which can be "<UNK>")
   - the secondary descriptor consists of a list of lists, because a secondary descriptor can be a set of multiple terms

## Iconclass

In [7]:
import iconclass

In [8]:
iconclass.get('25G41')

{'c': ['25G41(...)',
  '25G411',
  '25G412',
  '25GG41',
  '25G41(+0)',
  '25G41(+1)',
  '25G41(+2)',
  '25G41(+3)'],
 'txt': {'it': 'fiori',
  'pt': 'flores',
  'zh': '花卉',
  'nl': 'Bloemen….(met NAAM)',
  'de': 'Blumen',
  'fi': 'kukat',
  'fr': 'fleurs',
  'en': 'flowers'},
 'kw': {'it': ['fiore'],
  'pt': ['flor'],
  'nl': ['bloem'],
  'de': ['Blume'],
  'fi': ['kukka'],
  'fr': ['fleur'],
  'es': ['flor'],
  'en': ['flower']},
 'n': '25G41',
 'p': ['2', '25', '25G', '25G4', '25G41']}

From the [website](https://labs.brill.com/ictestset/):
    
> The keys in the returned dict are:
> - 'n' The notation of this node (the same thing as the first parameter of the .get() call)
> - 'p' Path to this node, starting from the root of the hierarchy
> - 'c' Children (descendants) of this node
> - 'kw' Keywords, a dict keyed on two-letter language code containing keywords associated with this entry
> - 'txt Textual descriptions of this notation, a dict of strings keyed on two-letter language code.

Idea:
    - get frequency and co-occurence data from Brill's iconclass dataset
    - map ontologies using fasttext and co-ocurence probabilities (some kind of beam search?)

In [9]:
import json
with open('../data/iconclass_img/data.json') as f:
    ic = json.loads(f.read())

In [10]:
len(ic)

87744

In [11]:
documents = []
for _, codes in ic.items():
    documents.append(codes)

In [12]:
print(len(documents))

87744


In [13]:
documents[:10]

[['31A235', '31A24(+1)', '61B(+54)', '61B:31A2212(+1)', '61B:31D14'],
 ['41D92', '25G41'],
 ['11H', '11I35', '11I36'],
 ['11F25', '11FF25', '41E2'],
 ['25F3', '25F38(PHEASANT)', '25H', '25H11', '25H213', '34B232(+51)'],
 ['22E12',
  '31AA2352',
  '31B52',
  '41C122',
  '41C326',
  '41C71',
  '54A43(+11)',
  '58AA1',
  '86(ESSEN, TRINKEN UND DAS SPIEL DER VENUS SCHADEN DORT, WO etc.)',
  '98B(SARDANAPALUS)'],
 ['25I1', '46C24', '25H13', '46C513', '48C161', '48C149', '25I151'],
 ['31D14', '41A2', '48C7311', '48C75'],
 ['31A231', '31A24(+1)', '61B(+55)', '61B:31A2212(+1)'],
 ['61B(+53)', '61B:31D14']]

In [14]:
for doc in documents[:10]:
    for term in doc:
        try:
            print(term, iconclass.get(term)['kw']['en'])
        except (KeyError, TypeError):
            pass

31A235 ['sitting']
31A24(+1) ['Stirnansicht', 'front view', 'head', 'posture', 'veduta frontale', 'vue de face']
61B(+54) ['Dreiviertel', 'three-quarter', 'tre quarti', 'trois-quarts']
41D92 ['woman']
25G41 ['flower']
11H ['saint']
11F25 ['Mater Dolorosa', 'bust', 'full length', 'half-length', 'head', 'mother']
41E2 ['optical illusion', 'picture', "trompe l'oeil"]
25F3 ['bird']
25F38(PHEASANT) ['pheasant']
25H ['ideal landscape', 'landscape']
25H11 ['mountain']
25H213 ['river']
34B232(+51) ['chicken', 'cock', 'fowl', 'hen', 'male', 'maschile', 'maschio', 'mâle', 'männlich']
22E12 ['burning']
31AA2352 ['seat']
31B52 ['alcohol', 'drunkenness']
41C122 ['pouring']
41C326 ['jar', 'jug']
41C71 ['alcohol', 'drink']
54A43(+11) ['Frau', 'Misura', 'Ripa', 'donna', 'femme', 'temperance', 'woman']
58AA1 ['destruction']
98B(SARDANAPALUS) ['Sardanapalus']
25I1 ['city', 'ideal city', 'veduta']
46C24 ['sailing-ship', 'yacht']
25H13 ['coast', 'sea']
46C513 ['traveller']
48C161 ['pillar']
48C149 ['ruin'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def identity(x):
    return x

vec = CountVectorizer(analyzer=identity, max_features=1000)
X = vec.fit_transform(documents).toarray()
print(X.shape)

In [ ]:
import pandas as pd

df = pd.DataFrame(X, columns=vec.get_feature_names())
df.head()

In [ ]:
df_asint = df.astype(int)
coocc = df_asint.T.dot(df_asint)
coocc.head()

In [ ]:
# https://stackoverflow.com/questions/58701337/how-to-construct-ppmi-matrix-from-a-text-corpus

import numpy as np

def pmi(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    # Silence distracting warnings about log(0):
    with np.errstate(divide='ignore'):
        df = np.log(df)
    df[np.isinf(df)] = 0.0  # log(0) = 0
    if positive:
        df[df < 0] = 0.0
    return df

In [ ]:
ppmi = pmi(coocc, positive=True)

In [ ]:
from itertools import combinations

pairs = []
for a, b in combinations(ppmi.index.tolist(), 2):
    pairs.append((ppmi.loc[a][b], a, b))

In [ ]:
pairs = sorted(pairs, reverse=True)

for sc, a, b in pairs[:200]:
    print('===================')
    try:
        print(iconclass.get(a)['kw']['en'], '&&&', iconclass.get(b)['kw']['en'], sc)
    except:
        pass

## Show labels

Alternatively, model documents with word2vec:

In [ ]:
for doc in documents[:10]:
    for term in doc:
        try:
            print(term, iconclass.get(term)['kw']['en'])
        except (KeyError, TypeError):
            pass

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(documents, size=50, window=5, min_count=1, workers=4)

In [ ]:
w = '41C326'
print(iconclass.get(w)['kw']['en'])
for t, sc in model.wv.most_similar(w):
    try:
        print(iconclass.get(t)['kw']['en'], sc)
    except KeyError:
        pass

In [ ]:
from collections import Counter
cnt = Counter()
for doc in documents:
    cnt.update(doc)
print(cnt.most_common(10))
voc = [w for w, _ in cnt.most_common(350)]
voc[:30]

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

%matplotlib inline

plt.style.use('dark_background')
cmap = plt.cm.get_cmap("cool")
from bokeh.palettes import Accent

def tsne_plot(model, voc, lang='en', num_clust=8):
    "Creates and TSNE model and plots it"
    labels, tokens = [], []

    for word in voc:
        tokens.append(model.wv[word])
        try:
            l = iconclass.get(word)['kw'][lang]
            l = '-'.join(l)[:15]
            labels.append(l)
        except:
            labels.append('$' + word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    coor = tsne_model.fit_transform(tokens)
    
    # run clustering on TSNE-coordinates:
    clustering = AgglomerativeClustering(linkage='ward',
                                     affinity='euclidean',
                                     n_clusters=num_clust)
    clustering.fit(coor)
    clust_labels = clustering.labels_
    colors = [Accent[num_clust][i] for i in clust_labels]
    
    # plot:
    fig, ax1 = plt.subplots(figsize=(14, 14)) # larger canvas
    x1, x2 = coor[:, 0], coor[:, 1]
    for a, b in zip(x1, x2):
        ax1.scatter(a, b, edgecolors='none', facecolors='none')
    for x, y, w, c in zip(x1, x2, labels, colors):
        if not w.startswith('$'):
            ax1.text(x, y, w, ha='center', va="center",
                 color=c,
                 fontdict={'family': 'Arial', 'size': 12})

    # control aesthetics:
    ax1.set_xlabel(''); ax1.set_ylabel('')
    ax1.set_xticklabels([]); ax1.set_xticks([])
    ax1.set_yticklabels([]); ax1.set_yticks([])
    plt.tight_layout()

    plt.show()

In [ ]:
tsne_plot(model, voc)

## FastText

In [130]:
path_fr = '../data/vectors/wiki.fr.align.vec'

from gensim.models.keyedvectors import KeyedVectors
wv_fr = KeyedVectors.load_word2vec_format(path_fr)

In [156]:
wv_fr.most_similar('chien')

[('chien…', 0.7481318116188049),
 ('chienne', 0.7443133592605591),
 ('chiens', 0.7350468039512634),
 ('chien»', 0.7269512414932251),
 ('chiens…', 0.7169510126113892),
 ('chat', 0.6941362619400024),
 ('chiot', 0.6886463165283203),
 ('dogue', 0.6747391819953918),
 ('caniche', 0.6535861492156982),
 ('aboie', 0.6478910446166992)]

In [158]:
iclass  = {}

for doc in documents:
    for code in doc:
        try:
            labels = iconclass.get(code)['kw']['fr']
            iclass[code] = tuple(labels)
        except (TypeError, KeyError):
            continue

iclass

{'31A235': ('asseoir',),
 '31A24(+1)': ('position', 'tête'),
 '41D92': ('femme',),
 '25G41': ('fleur',),
 '11H': ('saint',),
 '11F25': ('Mater Dolorosa', 'buste', 'en pied', 'mère', 'tête', 'à mi-corps'),
 '41E2': ("illusion d'optique", 'peinture', "trompe l'oeil"),
 '25F3': ('oiseau',),
 '25F38(PHEASANT)': ('faisan',),
 '25H': ('paysage', 'paysage idéal'),
 '25H11': ('montagne',),
 '25H213': ('rivière',),
 '34B232(+51)': ('coq', 'poule', 'poulet', 'volaille'),
 '22E12': ('brûler',),
 '31AA2352': ('siège',),
 '31B52': ('alcool', 'ivresse'),
 '41C122': ('verser',),
 '41C326': ('bocal', 'pichet'),
 '41C71': ('alcool', 'boisson'),
 '54A43(+11)': ('Misura', 'Ripa', 'tempérance'),
 '58AA1': ('destruction',),
 '98B(SARDANAPALUS)': ('Sardanapale',),
 '25I1': ('cité idéale', 'veduta', 'ville'),
 '46C24': ('voilier', 'yacht'),
 '25H13': ('côte', 'mer'),
 '46C513': ('voyageur',),
 '48C161': ('colonne',),
 '48C149': ('ruine',),
 '25I151': ('fontaine',),
 '31D14': ('adulte', 'homme'),
 '41A2': ('i

In [160]:
with open(f'../data/fabritius/json/fr.json', 'r', encoding='utf8') as json_file:
    instances = json.load(json_file)

fabritius = []
for record_id, iconography in list(instances.items())[:1000]:
    for dict_ in iconography:
        for top_term, sub_terms in dict_.items():
            tt =  [top_term]
            if sub_terms:
                for sub_term in sub_terms:
                    fabritius.append(tt + sub_term)
            else:
                fabritius.append(tt)

fabritius

[['effet de ligne'],
 ['figure', 'homme'],
 ['figure', 'barbe'],
 ['groupe de figures', 'chapeau', 'homme'],
 ['groupe de figures', 'assis', 'femme'],
 ['groupe de figures', 'pinceau'],
 ['intérieur', 'atelier'],
 ['intérieur', 'chevalet'],
 ['intérieur', 'tableau'],
 ['intérieur', 'toile'],
 ['intérieur', 'table'],
 ['intérieur', 'livre'],
 ['intérieur', 'fleur'],
 ['figure', 'en buste', 'femme'],
 ['figure', 'de profil'],
 ['figure', 'en pied', 'homme', 'nu'],
 ['figure', 'joint', 'main'],
 ['intérieur', 'ouvert', 'porte'],
 ['effet de ligne'],
 ['groupe de figures', 'homme'],
 ['groupe de figures', 'femme'],
 ['groupe de figures', 'foule'],
 ['groupe de figures', 'soldat'],
 ['groupe de figures', 'casque'],
 ['groupe de figures', 'arme', 'lance'],
 ['groupe de figures', 'animal', 'cheval', 'chien'],
 ['groupe de figures', 'coiffure'],
 ['groupe de figures', 'couronne', 'épine'],
 ['groupe de figures', 'nu'],
 ['abstrait'],
 ['effet de ligne'],
 ['figure', 'en pied', 'femme'],
 ['fig

In [161]:
print(len(fabritius))
print(len(iclass))

6007
18454


In [162]:
icodes = list(iclass.keys())
iclass = list(iclass.values())
all_docs = fabritius + iclass

In [163]:
#from gensim.models import FastText
#wv_fr = FastText(size=30, window=3, min_count=1, sentences=all_docs)
#wv_fr.wv.most_similar('temple')

In [164]:
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.corpora import Dictionary

In [165]:
termsim_index = WordEmbeddingSimilarityIndex(wv_fr)

In [166]:
dictionary = Dictionary(all_docs)

In [167]:
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)  # construct similarity matrix

In [168]:
bow_corpus = [dictionary.doc2bow(document) for document in iclass]
bow_corpus

[[(1046, 1)],
 [(61, 1), (1047, 1)],
 [(7, 1)],
 [(16, 1)],
 [(1048, 1)],
 [(19, 1), (46, 1), (61, 1), (628, 1), (1049, 1), (1050, 1)],
 [(1034, 1), (1051, 1), (1052, 1)],
 [(101, 1)],
 [(1053, 1)],
 [(71, 1), (1054, 1)],
 [(184, 1)],
 [(364, 1)],
 [(389, 1), (516, 1), (762, 1), (1055, 1)],
 [(1056, 1)],
 [(1057, 1)],
 [(291, 1), (1003, 1)],
 [(1058, 1)],
 [(1059, 1), (1060, 1)],
 [(972, 1), (1003, 1)],
 [(1061, 1), (1062, 1), (1063, 1)],
 [(1064, 1)],
 [(1065, 1)],
 [(172, 1), (1066, 1), (1067, 1)],
 [(1068, 1), (1069, 1)],
 [(40, 1), (1070, 1)],
 [(915, 1)],
 [(126, 1)],
 [(120, 1)],
 [(584, 1)],
 [(2, 1), (1071, 1)],
 [(10, 1), (73, 1)],
 [(402, 1)],
 [(111, 1), (842, 1)],
 [(68, 1), (655, 1)],
 [(1072, 1)],
 [(503, 1)],
 [(71, 1)],
 [(585, 1), (1073, 1), (1074, 1)],
 [(849, 1)],
 [(1075, 1)],
 [(123, 1)],
 [(1076, 1)],
 [(65, 1)],
 [(7, 1), (1071, 1)],
 [(1077, 1)],
 [(1078, 1)],
 [(37, 1)],
 [(47, 1)],
 [(61, 1), (1079, 1)],
 [(1080, 1)],
 [(230, 1), (721, 1), (722, 1), (1081, 1)]

In [174]:
from gensim.similarities import MatrixSimilarity, WmdSimilarity, SoftCosineSimilarity
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=10)

In [175]:
for query in fabritius[:100]:
    print('============')
    print(' '.join(query).split())
    q = dictionary.doc2bow(query)
    sims = docsim_index[q]
    for idx, score in sims:
        print('   ', score, ':', iclass[idx], icodes[idx])

['effet', 'de', 'ligne']
['figure', 'homme']
    0.7071067690849304 : ('homme',) 82A(VALERIO)
    0.7071067690849304 : ('Cérès', 'Ovide, Métamorphoses 05', 'Proserpine', 'Sirènes', 'chercher', 'compagnon', 'homme', 'mer') 97EE2
    0.7071067690849304 : ('adulte', 'homme') 31D14(+55)
    0.7071067690849304 : ('Lotis', 'Ovide, Métamorphoses 09', 'Priape', 'amour non partagé', 'fuir', 'homme', 'lotus') 97BB3
    0.7071067690849304 : ('Grecs', 'Romains', 'homme', 'identité', 'nom', 'personnage historique') 98B(PROTOGENES)
    0.7071067690849304 : ('homme',) 43C911
    0.7071067690849304 : ('homme',) 11I72
    0.7071067690849304 : ('adulte', 'homme') 31D14(+51)
    0.7071067690849304 : ('homme',) 82A(NUMA POMPILIUS)
    0.7071067690849304 : ('homme',) 82A(GUY OF WARWICK)
['figure', 'barbe']
    0.7071067690849304 : ('barbe', 'vieillard') 11C23(+31)
    0.7071067690849304 : ('barbe', 'droit (contraire de gauche)', 'gauche (contraire de droite)', 'poignard', 'poignarder') 71H8751
    0.707106

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sq

    0.7071067690849304 : ('chevalet',) 48C5151(+671)
    0.7071067690849304 : ('intérieur',) 51A51
    0.7071067690849304 : ('intérieur',) 41A3372
    0.7071067690849304 : ('intérieur',) 46C216
    0.7071067690849304 : ('intérieur',) 48C1412
    0.7071067690849304 : ('intérieur',) 41A3362
    0.7071067690849304 : ('intérieur',) 41A3362(+0)
    0.7071067690849304 : ('chevalet',) 44G321(RACK)
    0.7071067690849304 : ('intérieur',) 41C211
    0.7071067690849304 : ('intérieur',) 51A51(+4)
['intérieur', 'tableau']
    0.7071067690849304 : ('intérieur',) 48C1412
    0.7071067690849304 : ('intérieur',) 41A3372
    0.7071067690849304 : ('intérieur',) 51A51(+4)
    0.7071067690849304 : ('intérieur',) 41A3362
    0.7071067690849304 : ('intérieur',) 51A51
    0.7071067690849304 : ('intérieur',) 46C216
    0.7071067690849304 : ('intérieur', 'maison') 41A2
    0.7071067690849304 : ('intérieur',) 48C1426
    0.7071067690849304 : ('intérieur',) 41A3362(+0)
    0.7071067690849304 : ('intérieur', 'égl

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sq

    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+3)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+9225)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+9412)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+9224)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+935)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+823)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+9213)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+821)
    0.6079320311546326 : ('Corpo humano', 'Ripa', 'corps', 'femme', 'figure humaine', 'homme', 'nu') 31A(+72)


...

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sq

    0.7071067690849304 : ('activités', 'occupations humaines', 'paysage', 'saison', 'travail') 23E(+12)
    0.7071067690849304 : ('arbre',) 25G3(+37)
    0.7071067690849304 : ('paysage',) 25I(+1)
    0.7071067690849304 : ('abattage des arbres', 'abattre', 'activités', 'battage', 'cochon', 'cuire', 'douze', 'fauconnerie', 'fénaison', 'greffer', 'mois', 'moisson', 'occupations humaines', 'paysage', 'pressoir à vin', 'ramasser', 'récolte', 'semer', 'tonneau de vin', 'travail', 'vendange', 'vignoble', 'élaguer') 23K(+621)
    0.7071067690849304 : ('douze', 'mois', 'paysage') 23L(+61)
    0.7071067690849304 : ('arbre',) 25G3(+12)
    0.7071067690849304 : ('arbre', 'groupe') 25G11(+361)
    0.7071067690849304 : ('paysage', 'paysage idéal') 25K
    0.7071067690849304 : ('arbre',) 25G3(+24)
    0.7071067690849304 : ('paysage',) 11T411
['figure', 'homme', 'à', 'mi-corps']
    0.5773502588272095 : ('Cérès', 'Ovide, Métamorphoses 05', 'Proserpine', 'Sirènes', 'chercher', 'compagnon', 'homme', 'me

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sq

    0.7071067690849304 : ('Franciscains', "Louis d'Anjou (St)", 'Toulouse', 'chape', 'couronne', 'fleur-de-lis', 'sceptre', 'évêque') 11H(LOUIS OF TOULOUSE)
    0.513996958732605 : ('chape', 'dalmatique', 'pallium', 'rational', 'surplis', 'toge', 'tunicelle') 11Q71467(DALMATIC)
    0.513996958732605 : ('chape', 'dalmatique', 'pallium', 'rational', 'surplis', 'toge', 'tunicelle') 11Q71467(COPE)
    0.2842371463775635 : ('Thomas Becket (St)', 'archevêque', 'chasuble', 'crâne', 'martyr', 'rouge', 'épée') 11H(THOMAS BECKET)
    0.2842371463775635 : ('chasuble',) 11Q71463
    0.26322656869888306 : ('Jacques le Majeur (St)', 'apôtre', 'bourdon', 'chapeau', 'coquillage', 'coquille', 'habits de pèlerin', 'livre', 'manteau', 'rouleau', 'sac', 'épée') 11H(JAMES THE GREAT)(+0)
    0.26322656869888306 : ('bourdon', 'chapeau', 'gourde', 'habits de pèlerin', 'manteau', 'sac') 11Q624
    0.26322656869888306 : ('Jacques le Majeur (St)', 'apôtre', 'bourdon', 'chapeau', 'coquillage', 'coquille', 'habits

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sq